In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%writefile utils.py
def sn_ratio(model, X_test, y_test, name):
    """Compute quality losses for a given model."""
    y_pred = model.predict(X_test)

    # Compute performance metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"{name}")
    print(f"Rsquare score: {r2}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    # Compute quality losses
    L_r2 = 1 / ((r2 + 1e-9)**2) if r2 > 0 else 1
    L_mse = (1/2) * (mse ** 2)
    L_mae = (1/2) * (mae ** 2)

    
    return ({
        "Model": name,
        "R2 Loss": L_r2,
        "MSE Loss": L_mse,
        "MAE Loss": L_mae
    })
def process_mrsn(losses_df):
    for metric in ["R2 Loss", "MSE Loss", "MAE Loss"]:
    min_val, max_val = losses_df[metric].min(), losses_df[metric].max()
    if max_val != min_val:
        losses_df[f"Normalized {metric}"] = (losses_df[metric] - min_val) / (max_val - min_val)
    else:
        losses_df[f"Normalized {metric}"] = 0  # Avoid division by zero

    # Compute MRSN Scores
    epsilon = 1e-9
    for index, row in losses_df.iterrows():
        # Convert to numpy array explicitly and ensure float type
        normalized_values = np.array(row[["Normalized R2 Loss", "Normalized MSE Loss", "Normalized MAE Loss"]].values, dtype=float)
        # Replace zeros with epsilon
        normalized_values = np.maximum(normalized_values, epsilon)
        # Calculate SN ratios
        sn_ratios = -10 * np.log10(normalized_values)
        # Sum the ratios
        losses_df.loc[index, "MRSN Score"] = float(np.sum(sn_ratios))
    return losses_df

Writing utils.py
